In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/2014-Jan.csv', header=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471949 entries, 0 to 471948
Data columns (total 5 columns):
ORIGIN_AIRPORT_ID    471949 non-null int64
DEST_AIRPORT_ID      471949 non-null int64
ARR_DEL15            439620 non-null float64
CANCELLED            471949 non-null float64
Unnamed: 4           0 non-null float64
dtypes: float64(3), int64(2)
memory usage: 21.6 MB


In [3]:
df.columns = ['AirportID', 'Destination', 'Delayed', 'Cancelled', 'NONE']
df = df[['AirportID', 'Destination', 'Delayed', 'Cancelled']]
df = df[['AirportID', 'Delayed', 'Cancelled']]
df.head()

,AirportID,Delayed,Cancelled
0,10135,NaN,1
1,10135,0,0
2,10135,1,0
3,10135,0,0
4,10135,1,0


TODO:

Draw map-like bubble visualization where each bubble is a pie for cancellation/total.
Radius should represent total number of flights.

In [4]:
df.loc[df.Delayed.isnull(), 'Delayed'] = 0
df.head()

,AirportID,Delayed,Cancelled
0,10135,0,1
1,10135,0,0
2,10135,1,0
3,10135,0,0
4,10135,1,0


In [13]:
dfStat = df.groupby('AirportID', as_index=False).agg({'Cancelled': np.mean,
                                                      'Delayed': [np.mean, len]})
dfStat.columns = ['AirportID', 'Cancelled', 'Delayed', 'Count']
dfStat['Count'] = dfStat.Count.astype(int)
dfStat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 0 to 300
Data columns (total 4 columns):
(AirportID, )        301 non-null int64
(Cancelled, mean)    301 non-null float64
(Delayed, mean)      301 non-null float64
(Delayed, len)       301 non-null float64
dtypes: float64(3), int64(1)
memory usage: 11.8 KB


In [14]:
dfStat.head()

AirportID Cancelled   Delayed      
                 mean      mean   len
0     10135  0.162963  0.251852   135
1     10136  0.012346  0.156379   243
2     10140  0.034031  0.237173  1910
3     10141  0.032258  0.129032    62
4     10146  0.078652  0.134831    89

In [43]:
airports = pd.read_csv('data/airports.csv')
airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13094 entries, 0 to 13093
Data columns (total 9 columns):
AIRPORT_ID                        13094 non-null int64
AIRPORT                           13094 non-null object
DISPLAY_AIRPORT_NAME              13094 non-null object
DISPLAY_AIRPORT_CITY_NAME_FULL    13094 non-null object
AIRPORT_COUNTRY_NAME              13094 non-null object
LATITUDE                          13093 non-null float64
LONGITUDE                         13093 non-null float64
AIRPORT_IS_LATEST                 13094 non-null int64
Unnamed: 8                        0 non-null float64
dtypes: float64(3), int64(2), object(4)
memory usage: 1023.0+ KB


In [8]:
airports.rename(columns={'AIRPORT_ID': 'AirportID',
                         'LATITUDE': 'Lat',
                         'LONGITUDE': 'Lon',
                         'AIRPORT_IS_LATEST': 'IsLatest'}, inplace=True)
airports = airports.loc[airports.IsLatest == 1, ['AirportID', 'Lat', 'Lon']]
airports.head()

,AirportID,Lat,Lon
0,10001,58.109444,-152.906667
1,10003,65.548056,-161.071667
2,10004,68.083333,-163.166667
3,10005,67.570000,-148.183889
4,10006,57.745278,-152.882778


In [9]:
airports[airports.AirportID == 10135]

,AirportID,Lat,Lon
156,10135,40.652222,-75.440556


In [39]:
joinedDF = pd.merge(dfStat, airports, on='AirportID')
joinedDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 0 to 300
Data columns (total 6 columns):
AirportID    301 non-null int64
Delay        301 non-null float64
Cancelled    301 non-null float64
Count        301 non-null int64
Lat          301 non-null float64
Lon          301 non-null float64
dtypes: float64(4), int64(2)
memory usage: 16.5 KB


In [40]:
joinedDF.tail()

,AirportID,Delay,Cancelled,Count,Lat,Lon
296,15624,0.228856,0.136816,402,30.483333,-86.525278
297,15841,0.112903,0.048387,62,56.484444,-132.369722
298,15919,0.257869,0.128329,826,36.281667,-94.307778
299,15991,0.133333,0.000000,60,59.503333,-139.660278
300,16218,0.075758,0.018939,264,32.656667,-114.605833


In [41]:
joinedDF.to_csv('flights-stat.csv', index=False)

In [42]:
joinedDF.sort('Lon')

,AirportID,Delay,Cancelled,Count,Lat,Lon
7,10165,0.111111,0.000000,9,51.878056,-176.646111
230,14222,0.100000,0.000000,10,-14.330000,-170.710278
213,13873,0.133333,0.033333,60,64.512500,-165.444444
219,13970,0.200000,0.016667,60,66.884722,-162.598056
25,10551,0.144578,0.036145,83,60.778611,-161.837222
171,12982,0.071346,0.004603,869,21.976111,-159.338889
134,12173,0.061787,0.003447,3771,21.317778,-157.920278
43,10754,0.230769,0.012821,78,71.284722,-156.768611
210,13830,0.080756,0.003436,1746,20.898611,-156.430556
158,12758,0.069613,0.006630,905,19.738889,-156.045556


In [44]:
airports[airports.AIRPORT_ID == 12016]

,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_COUNTRY_NAME,LATITUDE,LONGITUDE,AIRPORT_IS_LATEST,Unnamed: 8
4133,12016,GUM,Agana Field,"Guam, Guam",Guam,13.483889,144.797222,0,NaN
4134,12016,GUM,Guam International,"Guam, TT",United States,13.483889,144.797222,1,NaN
